# Projet Air-Bnb

# 1) Problématique

Au cours des derniers mois, la France a été frappée par le virus du COVID 19. Cette pandémie s'est répandue rapidement au niveau national, avec de grandes disparités entre les régions. Le gouvernement français met en place un système d'information sur l'état de contamination au niveau départemental. Le 28 mai, L'Île de France est notée "zone rouge", ce qui signifie qu'elle reste encore fortement touchée par le virus, dans le même temps, des régions se déconfinent progressivement. Aujourd'hui, la règle des "100 km" ayant été relevée, il est désormais possible de se déplacer sur l'ensemble du territoire.

Cette actualité a nourri beaucoup de réflexions sur le travail à distance, largement favorisée par les entreprises afin de limiter les risques de contamination. 

Problématique : 

Didier est Data Scientist au sein du service analyse quantitative de l'AP-HP. Il est l'heureux propriétaire de deux biens immobiliers. Sa résidence principale se situe à Paris, sa résidence secondaire à Nice. Didier souhaiterait mettre l'un de ces deux appartements en location, sur AirBNB, pour la période estivale. Lequel de ses logements a t-il intérêt à mettre en location cet été ? 

Didier est en télé-travail, il peut donc travailler depuis Paris ou Nice et il est indifferent entre ces deux villes. On suppose que Didier louera son logement là où il maximisera son profit. 

Pour aider Didier à prendre sa décision, nous avons scappé les données d'Air-Bnb pour ces deux villes. Après avoir nettoyé la base, nous sélectionnerons les variables qui caractérisent le mieux le prix par nuit pour les deux appartements.

# 2) Packages utilisés

Pour rendre l'expérience utilisateur plus simple et présenter des résultats lisible, nous avons packagé toutes les fonctions que nous utilisons. Merci de les importer en modifiant l'os.chdir indiqué.

In [1]:
import os

os.chdir("Le path du projet pour importer les modules")

import pandas as pd
from Packages.Search_air_bnb import Search
from Packages.Scrap_Air_Bnb import Scrap_Air_Bnb
from Packages.Clean_Air_Bnb import cleaning
from Packages.Var_selection import corr_select
from Packages.Var_selection import pvalue_select
from Packages.Var_selection import ACP_select
from Packages.Var_selection import rfe_select
from Packages.Modelisation import modelisation

# 3) Partie Scraping

Nous avons construit des fonctions automatisant le scraping d'Air-Bnb dans le but de rendre notre projet utilisable sur plusieurs villes. On peut grace à la fonction Search indiquer le lieu, les dates et le nombre de voyageurs que l'on désire afin d'obtenir un lien qui servira de base au web-scraping.

La fonction Scrap_Air_Bnb lance le scraping, on peut préciser le nombre page que l'on désire scrapper, correspondant à vingt logements.

Si vous désirez lancer le scraping, merci de remplacer le path qui indique le Chrome Driver nécéssaire pour utiliser sélenium.

Les bases finales que nous avons mis à votre disposition sont une compilation de plusieurs bases récupérées en scrappant sur plusieur périodes de la durée estival. Le maximum que l'on puisse scraper en une seule fois est 15*20= 300 observations a cause de la structure du site d'Air Bnb.

In [ ]:
path=r"Le path du webdriver\chromedriver.exe"

link=Search(chrome_path=path, Ville="Nice",Arrivée_Mois=7, Arrivée_Jour=1, Départ_Mois=7, Départ_Jour=15, Adultes=1, Enfants=0, Bébés=1)

Base=Scrap_Air_Bnb(chrome_path=path, lien=link, Nb_vingtaine_pages=5)

Base.to_csv("Votre dossier//Base.csv")

# 4) Partie Cleaning

Après avoir scrapé, obtenu nos données brutes, puis réimporté la base, nous avons automatisé le nettoyage de la base en supprimant les valeurs manquante et en supprimant les données abérrantes.

Vous pouvez commencer ici a importer la base fournie en l'important et en la nettoyant.

In [2]:
Base=pd.read_csv(r'Le path de la base\\Paris.csv')

Clean_Base=cleaning(DataFrame=Base)

# 5) Partie Selection et Réduction 

Nous utilisons plusieurs méthodes pour sélectioner des variables.
Premierement nous avons réduit le nombre de variables en faisant de l'Analyse en Composantes Principales (beaucoup de variables sont trés corrélées, de plus les équipements sont plus intéressants exploités en lot). Nous avons ensuite étudié les corrélations, la significativité de nos variables et fait de la RFE (Recursive Feature Elimnation).

## 5.1) ACP

Ici on effectue de l'ACP pour créer des variables synthétiqes qui pourront etre incorporées dans la modélisation.

On obtient les variables Cuisine1, Secu1, Host1, Bébé1, Luxe1, List_nb. On les ajoute a notre base. On crée aussi une base annexe qui contient juste le prix et les variables tirées de l'ACP.

In [3]:
acp=ACP_select(Clean_Base)

Clean_Base_plus_acp=pd.concat([Clean_Base,acp], axis=1).dropna()

Clean_Base_only_acp=pd.concat([Clean_Base["Prix_sans_reduc"],acp], axis=1).dropna()

print(acp.columns)

Index(['Cuisine1', 'Secu1', 'Host1', 'Bébé1', 'Luxe1', 'List_nb'], dtype='object')


## 5.2) Sélection par corrélations

Ici nous analysons les corrélations entre nos variables et le prix. La fonction corr_select que nous avons crée nous rend une DatFrame contenant les variables les plus corrélées (selon le niveau de corrélation précisé dans la fonction, on prend les variables corrélées au delà de ce niveau) et une liste des variables les plus corrélées.

In [4]:
Resultat_Clean_Base=corr_select(Clean_Base,Var="Prix_sans_reduc",corr=0.2)

Resultat_Clean_Base_plus_acp=corr_select(Clean_Base_plus_acp,Var="Prix_sans_reduc",corr=0.3)


Basic_df_corr_selected=Resultat_Clean_Base[0]
Basic_list_corr=Resultat_Clean_Base[1]


ACP_df_corr_selected=Resultat_Clean_Base_plus_acp[0]
ACP_list_corr=Resultat_Clean_Base_plus_acp[1]

print(Basic_list_corr)

print(ACP_list_corr)

['lave_linge', 'Fer_à_Repasser', 'Télévision', 'Lit_pour_bébé', 'Lave_vaisselle', 'Cafetière', 'Cuisinière', 'Sèche_cheveux', 'Bébés', 'Nb_équipement', 'Prix_sans_reduc', 'Nb_Voyageurs', 'Nb_Lits', 'Nb_Salles_de_Bain', 'Studio']
['Lave_vaisselle', 'Nb_équipement', 'Prix_sans_reduc', 'Nb_Voyageurs', 'Nb_Lits', 'Nb_Salles_de_Bain', 'Studio', 'List_nb']


## 5.3) Sélection par P-value

Toujours dans le but de trouver des variables intéressantes, nous utilisons ici stats model et testonn toutes les variables présente s dans la base indiquée. Nous affichons ensuite les variables triées par significativité.

In [5]:
All_Variables=Clean_Base.columns.astype(list)
All_Variables_X=All_Variables.drop(['Prix_sans_reduc','Prix_reduc'])

All_Variables_acp=Clean_Base_plus_acp.columns.astype(list)
All_Variables_X_acp=All_Variables_acp.drop(['Prix_sans_reduc','Prix_reduc'])


P_values=pvalue_select(Clean_Base,X=All_Variables_X)
P_values_acp=pvalue_select(Clean_Base_plus_acp,X=All_Variables_X_acp)

P_values.drop(["const"],inplace=True)
P_values_acp.drop(["const"],inplace=True)

P_values.sort_values(ascending=True, inplace=True)
print(P_values.head(20))

P_values_acp.sort_values(ascending=True, inplace=True)
print(P_values_acp.head(20))



Nb_Voyageurs                             4.473211e-46
Nb_Salles_de_Bain                        3.637855e-17
Jardin                                   1.815755e-05
coté                                     5.875264e-05
Annulation_gratuite_pendant_48_heures    1.069737e-04
Annulation_gratuite_longue               2.460499e-04
Lave_vaisselle                           9.257754e-04
Note                                     9.294789e-04
Nb_comments                              1.951732e-03
Détecteur_de_fumée                       3.440163e-03
Kit_de_premiers_secours                  8.613873e-03
Superhost_list                           1.067289e-02
Nb_Lits                                  1.112722e-02
Hôte_expérimenté                         1.154361e-02
Jacuzzi                                  1.625775e-02
Logement_Entier                          1.984050e-02
Nb_équipement                            2.048462e-02
Extincteur                               2.342788e-02
Arrivée_irréprochable       

In [6]:
P_value_Base=Clean_Base[P_values.index[0:10]]

P_value_ACP_Base=Clean_Base_plus_acp[P_values_acp.index[0:10]]

## 5.4) Sélection par RFE

Enfin, une autre façon de sélectionner des variables est d'utiliser la RFE. Dans cette fonction on peut indiquer le modèle utilisé pour la RFE ("LinearRegression","DecisionTreeRegressor","RandomForestRegressor" ou "LinearSVR"). On indique également le nombre de variables souhaitées et la base.

In [7]:
Base_RFE=rfe_select (Base=Clean_Base, nb_var=20, model="LinearRegression")
Base_RFE_ACP=rfe_select (Base=Clean_Base_plus_acp, nb_var=20, model="LinearRegression")

print(Base_RFE.columns)
print(Base_RFE_ACP.columns)

C:\Users\Yanis\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Yanis\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Index(['Note', 'Logement_Entier', 'Hôte_expérimenté', 'Superhost_list',
       'Wi-Fi', 'Parking_payant', 'Chargeur_EV', 'Jacuzzi', 'Cuisine', 'four',
       'Lave_vaisselle', 'Cafetière', 'Ustensiles_de_cuisine_de_base',
       'Acceuil_Personnel', 'Jardin', 'Extincteur', 'Nb_Voyageurs', 'Nb_Lits',
       'Nb_Salles_de_Bain', 'Prix_sans_reduc'],
      dtype='object')
Index(['Note', 'Logement_Entier', 'Arrivée_irréprochable', 'Superhost_list',
       'Wi-Fi', 'Lit_pour_bébé', 'Parking_gratuit', 'Jacuzzi', 'Cuisine',
       'four', 'Lave_vaisselle', 'Cafetière', 'Ustensiles_de_cuisine_de_base',
       'Acceuil_Personnel', 'Salon_privé', 'Jardin', 'Extincteur',
       'Nb_Voyageurs', 'Nb_Lits', 'Prix_sans_reduc'],
      dtype='object')


# 6) Modélisation 

## 6.1) Indicateur de performance sur différents modèles 

Pour la modélisation, on a comparé dans un premier temps les RMSE issues de la cross validation de plusieurs modèles. La fonction ci dessous nous indique les RMSE moyennes pour les modeles suivants : régression linéaire, arbre de décision, random forest, SVM linéaire et SVM polynomial.

In [8]:
Clean_Base.drop(['Prix_reduc'],axis=1,inplace=True)
modelisation(Clean_Base)

LiR: 27.836942 (2.080079)
DTR: 39.871937 (2.944525)
RDF: 29.049918 (1.979575)
SVM: 28.575318 (2.014332)
SVR: 36.649379 (3.450931)


In [9]:
Clean_Base_plus_acp.drop(['Prix_reduc'],axis=1,inplace=True)
modelisation(Clean_Base_plus_acp)

LiR: 27.286693 (1.513712)
DTR: 39.331085 (3.374562)
RDF: 28.354743 (1.741343)
SVM: 28.067786 (1.860659)
SVR: 34.676661 (2.796816)


In [10]:
modelisation(Clean_Base_only_acp)

LiR: 34.172508 (2.818248)
DTR: 48.858917 (2.190641)
RDF: 38.235704 (2.338898)
SVM: 35.391979 (2.568066)
SVR: 41.803907 (3.920369)


In [11]:
modelisation(ACP_df_corr_selected)

LiR: 28.238760 (1.453340)
DTR: 39.339255 (2.672288)
RDF: 32.307376 (2.028051)
SVM: 29.034316 (1.790081)
SVR: 33.761286 (1.629750)


## 6.2) Regression Linéaire

Les résultats que nous trouvons les plus intéressants concernent le modèle de régression linéaire. Nous décidons de mettre en oeuvre cette régression pour prédir le prix

## 6.2.1) Modèle complet

In [13]:
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse

Test=Clean_Base.copy()

try:
    Test.drop(['Prix_reduc'],axis=1,inplace=True)
except:
    pass

y=Test["Prix_sans_reduc"]
x=Test.drop(["Prix_sans_reduc"],axis=1)
x=sm.add_constant(x)
model=sm.OLS(y,x)
results = model.fit()
results.summary()
ypred=results.predict(x)

rmse=rmse(y, ypred)

print("Rmse ="+str(rmse))
print(results.summary())

Rmse =26.381086314497033
                            OLS Regression Results                            
Dep. Variable:        Prix_sans_reduc   R-squared:                       0.654
Model:                            OLS   Adj. R-squared:                  0.638
Method:                 Least Squares   F-statistic:                     40.55
Date:                Sat, 06 Jun 2020   Prob (F-statistic):          1.39e-297
Time:                        23:16:03   Log-Likelihood:                -7483.1
No. Observations:                1595   AIC:                         1.511e+04
Df Residuals:                    1523   BIC:                         1.550e+04
Df Model:                          71                                         
Covariance Type:            nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------

## 6.2.2) Modèle complet plus ACP

In [14]:
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse

Test=Clean_Base_plus_acp.copy()

try:
    Test.drop(['Prix_reduc'],axis=1,inplace=True)
except:
    pass

y=Test["Prix_sans_reduc"]
x=Test.drop(["Prix_sans_reduc"],axis=1)
x=sm.add_constant(x)
model=sm.OLS(y,x)
results = model.fit()
results.summary()
ypred=results.predict(x)

rmse=rmse(y, ypred)

print("Rmse ="+str(rmse))
print(results.summary())

Rmse =25.550754053218935
                            OLS Regression Results                            
Dep. Variable:        Prix_sans_reduc   R-squared:                       0.638
Model:                            OLS   Adj. R-squared:                  0.620
Method:                 Least Squares   F-statistic:                     33.81
Date:                Sat, 06 Jun 2020   Prob (F-statistic):          1.16e-269
Time:                        23:16:20   Log-Likelihood:                -7231.7
No. Observations:                1552   AIC:                         1.462e+04
Df Residuals:                    1474   BIC:                         1.504e+04
Df Model:                          77                                         
Covariance Type:            nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------

## 6.2.3) Modèle juste ACP

In [15]:
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse

Test=Clean_Base_only_acp.copy()

try:
    Test.drop(['Prix_reduc'],axis=1,inplace=True)
except:
    pass

y=Test["Prix_sans_reduc"]
x=Test.drop(["Prix_sans_reduc"],axis=1)
x=sm.add_constant(x)
model=sm.OLS(y,x)
results = model.fit()
results.summary()
ypred=results.predict(x)

rmse=rmse(y, ypred)

print("Rmse ="+str(rmse))
print(results.summary())

Rmse =34.56351559866377
                            OLS Regression Results                            
Dep. Variable:        Prix_sans_reduc   R-squared:                       0.338
Model:                            OLS   Adj. R-squared:                  0.336
Method:                 Least Squares   F-statistic:                     131.7
Date:                Sat, 06 Jun 2020   Prob (F-statistic):          8.20e-135
Time:                        23:16:31   Log-Likelihood:                -7700.6
No. Observations:                1552   AIC:                         1.542e+04
Df Residuals:                    1545   BIC:                         1.545e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         82.4654      0

## 6.2.4) Modèle Corr-select

In [16]:
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse

Test=Basic_df_corr_selected.copy()

try:
    Test.drop(['Prix_reduc'],axis=1,inplace=True)
except:
    pass

y=Test["Prix_sans_reduc"]
x=Test.drop(["Prix_sans_reduc"],axis=1)
x=sm.add_constant(x)
model=sm.OLS(y,x)
results = model.fit()
results.summary()
ypred=results.predict(x)

rmse=rmse(y, ypred)

print("Rmse ="+str(rmse))
print(results.summary())

Rmse =28.779308730346596
                            OLS Regression Results                            
Dep. Variable:        Prix_sans_reduc   R-squared:                       0.588
Model:                            OLS   Adj. R-squared:                  0.585
Method:                 Least Squares   F-statistic:                     161.2
Date:                Sat, 06 Jun 2020   Prob (F-statistic):          5.03e-292
Time:                        23:16:36   Log-Likelihood:                -7621.9
No. Observations:                1595   AIC:                         1.527e+04
Df Residuals:                    1580   BIC:                         1.535e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const        

## 6.2.5) Modèle Corr select plus ACP

In [17]:
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse

Test=ACP_df_corr_selected.copy()

try:
    Test.drop(['Prix_reduc'],axis=1,inplace=True)
except:
    pass

y=Test["Prix_sans_reduc"]
x=Test.drop(["Prix_sans_reduc"],axis=1)
x=sm.add_constant(x)
model=sm.OLS(y,x)
results = model.fit()
results.summary()
ypred=results.predict(x)

rmse=rmse(y, ypred)

print("Rmse ="+str(rmse))
print(results.summary())

Rmse =28.479512783125735
                            OLS Regression Results                            
Dep. Variable:        Prix_sans_reduc   R-squared:                       0.551
Model:                            OLS   Adj. R-squared:                  0.549
Method:                 Least Squares   F-statistic:                     270.5
Date:                Sat, 06 Jun 2020   Prob (F-statistic):          4.92e-263
Time:                        23:16:42   Log-Likelihood:                -7400.1
No. Observations:                1552   AIC:                         1.482e+04
Df Residuals:                    1544   BIC:                         1.486e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const        

## 6.2.6) Modèle P-value 

In [18]:
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse

Test=P_value_Base.copy()
Test=pd.concat([Test,Clean_Base["Prix_sans_reduc"]], axis=1).dropna()

try:
    Test.drop(['Prix_reduc'],axis=1,inplace=True)
except:
    pass

y=Test["Prix_sans_reduc"]
x=Test.drop(["Prix_sans_reduc"],axis=1)
x=sm.add_constant(x)
model=sm.OLS(y,x)
results = model.fit()
results.summary()
ypred=results.predict(x)

rmse=rmse(y, ypred)

print("Rmse ="+str(rmse))
print(results.summary())

Rmse =28.235209861662607
                            OLS Regression Results                            
Dep. Variable:        Prix_sans_reduc   R-squared:                       0.604
Model:                            OLS   Adj. R-squared:                  0.601
Method:                 Least Squares   F-statistic:                     241.3
Date:                Sat, 06 Jun 2020   Prob (F-statistic):          9.67e-310
Time:                        23:16:47   Log-Likelihood:                -7591.4
No. Observations:                1595   AIC:                         1.520e+04
Df Residuals:                    1584   BIC:                         1.526e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------

## 6.2.7) Modèle P-value plus ACP

In [19]:
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse

Test=P_value_ACP_Base.copy()
Test=pd.concat([Test,Clean_Base["Prix_sans_reduc"]], axis=1).dropna()

try:
    Test.drop(['Prix_reduc'],axis=1,inplace=True)
except:
    pass

y=Test["Prix_sans_reduc"]
x=Test.drop(["Prix_sans_reduc"],axis=1)
x=sm.add_constant(x)
model=sm.OLS(y,x)
results = model.fit()
results.summary()
ypred=results.predict(x)

rmse=rmse(y, ypred)

print("Rmse ="+str(rmse))
print(results.summary())

Rmse =28.091010884785838
                            OLS Regression Results                            
Dep. Variable:        Prix_sans_reduc   R-squared:                       0.563
Model:                            OLS   Adj. R-squared:                  0.560
Method:                 Least Squares   F-statistic:                     198.6
Date:                Sat, 06 Jun 2020   Prob (F-statistic):          1.39e-268
Time:                        23:16:51   Log-Likelihood:                -7378.8
No. Observations:                1552   AIC:                         1.478e+04
Df Residuals:                    1541   BIC:                         1.484e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------

## 6.2.8) Modèle RFE

In [20]:
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse

Test=Base_RFE.copy()

try:
    Test.drop(['Prix_reduc'],axis=1,inplace=True)
except:
    pass

y=Test["Prix_sans_reduc"]
x=Test.drop(["Prix_sans_reduc"],axis=1)
x=sm.add_constant(x)
model=sm.OLS(y,x)
results = model.fit()
results.summary()
ypred=results.predict(x)

rmse=rmse(y, ypred)

print("Rmse ="+str(rmse))
print(results.summary())

Rmse =27.835248444756786
                            OLS Regression Results                            
Dep. Variable:        Prix_sans_reduc   R-squared:                       0.615
Model:                            OLS   Adj. R-squared:                  0.610
Method:                 Least Squares   F-statistic:                     132.3
Date:                Sat, 06 Jun 2020   Prob (F-statistic):          2.90e-309
Time:                        23:16:56   Log-Likelihood:                -7568.7
No. Observations:                1595   AIC:                         1.518e+04
Df Residuals:                    1575   BIC:                         1.528e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------

## 6.2.7) Modèle RFE plus ACP

In [21]:
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse

Test=Base_RFE_ACP.copy()

try:
    Test.drop(['Prix_reduc'],axis=1,inplace=True)
except:
    pass

y=Test["Prix_sans_reduc"]
x=Test.drop(["Prix_sans_reduc"],axis=1)
x=sm.add_constant(x)
model=sm.OLS(y,x)
results = model.fit()
results.summary()
ypred=results.predict(x)

rmse=rmse(y, ypred)

print("Rmse ="+str(rmse))
print(results.summary())

Rmse =27.669062085506372
                            OLS Regression Results                            
Dep. Variable:        Prix_sans_reduc   R-squared:                       0.576
Model:                            OLS   Adj. R-squared:                  0.571
Method:                 Least Squares   F-statistic:                     109.6
Date:                Sat, 06 Jun 2020   Prob (F-statistic):          8.81e-269
Time:                        23:17:01   Log-Likelihood:                -7355.3
No. Observations:                1552   AIC:                         1.475e+04
Df Residuals:                    1532   BIC:                         1.486e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------

## 6.3) Prédiction de Prix

Pour prédire un prix nous allons renseigné la base de la ville qui nous intéresse, nous créons une observation correspondant à l'appartement de didier, nous estimons le modele sur notre base puis nous appliquont le modèle a l'appartement. Dans cette premiere étépe nous créons l'appartement de didier et on lui calcule les variables d'acp qui lui revienne.

In [22]:
Base=pd.read_csv(r'Chemin de la base\\Paris.csv')
Clean_Base=cleaning(DataFrame=Base)

Prix_sans_reduc=Clean_Base["Prix_sans_reduc"].mean()
Prix_reduc=Clean_Base["Prix_reduc"].mean()

observation=['4.612234','1','1','0','1','0','1','1','1','1','1','1','0','1','0','0','1','1','1','1','1','1','0','0','0','0','1','0','0','0','0','0','0','1','1','1','1','0','1','1','1','1','1','0','0','0','1','0','0','0','1','1','1','1','1','0','1','1','1','0','0','0','0','0','0','0','24','100','100','34','2','1','1','0','1','1','0']
my_appart_carac=[float(i) for i in observation]
Clean_Base.reset_index(drop=True,inplace=True)
Clean_Base.loc[len(Clean_Base)]=my_appart_carac

Clean_Base.loc[len(Clean_Base)-1,"Prix_sans_reduc"]=Prix_sans_reduc
Clean_Base.loc[len(Clean_Base)-1,"Prix_reduc"]=Prix_reduc

acp=ACP_select(Clean_Base)
Clean_Base_plus_acp=pd.concat([Clean_Base,acp], axis=1).dropna()

New_Obs=Clean_Base_plus_acp.iloc[len(Clean_Base_plus_acp)-1]

Ensuite, nous estimons le modèle sur notre base, nous choisissons le modèle corrélation plus acp car il arrive à des scores de RMSE correct et est plus interpetrétable qu'un modèle trop complet 

In [23]:
Base=pd.read_csv(r' Chemin de la base\\Paris.csv')
Clean_Base=cleaning(DataFrame=Base)

acp=ACP_select(Clean_Base)
Clean_Base_plus_acp=pd.concat([Clean_Base,acp], axis=1).dropna()
Resultat_Clean_Base_plus_acp=corr_select(Clean_Base_plus_acp,Var="Prix_sans_reduc",corr=0.25)
ACP_df_corr_selected=Resultat_Clean_Base_plus_acp[0]


import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse

Test=ACP_df_corr_selected.copy()

try:
    Test.drop(['Prix_reduc'],axis=1,inplace=True)
except:
    pass

y=Test["Prix_sans_reduc"]
x=Test.drop(["Prix_sans_reduc"],axis=1)
x=sm.add_constant(x)
model=sm.OLS(y,x)
results = model.fit()

ypred=results.predict(x)
rmse=rmse(y, ypred)

print("RMSE ="+str(rmse))
print(results.summary())

RMSE =28.28789535891183
                            OLS Regression Results                            
Dep. Variable:        Prix_sans_reduc   R-squared:                       0.557
Model:                            OLS   Adj. R-squared:                  0.554
Method:                 Least Squares   F-statistic:                     193.7
Date:                Sat, 06 Jun 2020   Prob (F-statistic):          6.29e-264
Time:                        23:17:18   Log-Likelihood:                -7389.7
No. Observations:                1552   AIC:                         1.480e+04
Df Residuals:                    1541   BIC:                         1.486e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const         

Enfin on présente notre prix estimé (ici pour Paris) ainsi qu'un intervalle de confiance

In [24]:
Appartement=New_Obs[x.columns]
Appartement["const"]=1

App2=Appartement.to_frame()
New_location=App2.transpose()

ypred=results.predict(New_location)


import numpy as np
import scipy.stats

yres=results.predict(x)


def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return h


int_conf=mean_confidence_interval(yres)

print("le prix prédit est de "+str(int(ypred.iloc[0]))+"€ pour la location à Paris")
print ("avec un intervalle confiance de +/- :" +str(int_conf))


le prix prédit est de 66€ pour la location à Paris
avec un intervalle confiance de +/- :1.579450159193012


Pour Nice le pris est de 71, le prix du logement à Nice est plus cher que celui qu'il a à Paris. Il est donc plus rentable pour lui de louer son appartement au Sud.